# Comparison of German economic Professors that signed the Hamburger Appell

#### Der Hamburger Appell

Der Hamburger Appell ist ein kurz vor der Bundestagswahl 2005 von 243 Wirtschaftswissenschaftlern [1] unterzeichneter, öffentlicher Appell für wirtschaftspolitische Reformen in Deutschland. Der Appell wurde durch eine Anzeigenserie der Initiative Neue Soziale Marktwirtschaft (INSM), in welcher der Hamburger Appell auszugsweise zitiert wurde, in die breite Öffentlichkeit getragen.

In dem Appell werden fiskalische Ausgabenprogramme als grundsätzlich untauglich eingestuft. Die gesamtwirtschaftliche Nachfrage sei „eine bedeutende und komplex strukturierte ökonomische Größe, die sich einer nachhaltigen Steuerung weitestgehend entzieht.“ Deshalb müsse „eine verantwortungsbewusste Finanzpolitik streng stabilitätsorientiert sein.“ [5] Konkret wurde u.a. gefordert, dass die Geringverdienenden weniger verdienen sollten und die Sozialhilfe vermehrt von Lohnersatzleistungen zu Lohnzuschüssen wechseln müsse. (Source:Wikipedia)

In [20]:
import pandas as pd
import Levenshtein
import difflib
import regex as re

**The list contains supervisory and advisory-memberships of think tanks, which have pushed for auterity politics after financial crisis. See: http://bit.ly/2fcjGwT**

In [21]:
Austerity_Path = '~/Csv_Dateien/thinktanks.csv'
Austerity_Members = pd.read_csv(Austerity_Path, encoding = "ISO-8859-1", sep=";")

**Comparison to Professors that have signed The Hamburger Appell**

In [22]:
df_Path = '~/Csv_Dateien/HamburgerAppell.csv'
df = pd.read_csv(df_Path,encoding ="ISO-8859-1", sep=";")




**Strip trailing spaces:**

In [23]:
df['Namen'] = df['Namen'].map(lambda x: x.strip())

**drop duplicate entries**

In [24]:
df = df.drop_duplicates()

**separate Listentries by substring 'Dr.'**

In [25]:
#dfname = df.drop(df.columns[[2,3,4]], axis=1)
dfname = df
dfname['Title'],dfname['Name'] = zip(*df.Namen.apply(lambda x: x.split('Dr.')))
df = dfname.drop(dfname.columns[[0,2,3]], axis=1)
df.head()

,Forschungseinrichtung,Name
0,Universit_t G_ttingen,Frank Achtenhagen
1,Universit_t Hohenheim,Michael Ahlheim
2,Universit_t des Saarlandes,Max Albert
3,Universit_t Essen,Erwin Amann
4,Universit_t Duisburg,Peter Anker


**Cleaning the dataframe. Change Headings and replace commma wih semicolon**

In [26]:
for a in df['Name']:
    if "," in a:
        df['Name'] = df["Emp name"].str.replace(",",";")
    #if re.findall(',\s',) not in a:
        #print("no")
      
df['Name'] = df['Name'].str.replace(r'\;([a-zA-Z])',r'; \1')


**check the dataframe of economists that signed the Hamburger Appell**

In [27]:
df.head()

,Forschungseinrichtung,Name
0,Universit_t G_ttingen,Frank Achtenhagen
1,Universit_t Hohenheim,Michael Ahlheim
2,Universit_t des Saarlandes,Max Albert
3,Universit_t Essen,Erwin Amann
4,Universit_t Duisburg,Peter Anker


**check dataframe of Advisory/Supervisory-Board of think tanks**

In [28]:
Austerity_Members.head()

,Emp name,Thinktank
0,EdwinMeese III,Adriatic Institute for Public Policy
1,JohnBlundell,Adriatic Institute for Public Policy
2,MikeBubalo,Adriatic Institute for Public Policy
3,EamonnButler,Adriatic Institute for Public Policy
4,HelenDisney,Adriatic Institute for Public Policy


**check if indidivdual that signed Hamburger Appell holds think tank membership. 
Does not only account for direct matches but takes similar names into account (jaro_score).** 

In [29]:
Austerity_and_Potential = []

for name in df['Name']:
    Potential = []
    Member = []
    for aust in Austerity_Members['Emp name']:
        aust = str(aust)
        distance = Levenshtein.distance(name,aust)
        jaro = Levenshtein.jaro_winkler(name,aust)
        if name == aust:
            Member.append(name)
            Member.append(aust)
            Member.append(jaro)
        
        elif distance <= 3 and distance != 0 and jaro>=0.80:
            Potential.append(name)
            Potential.append(aust)
            Potential.append(jaro)
    if Member != []:
        Austerity_and_Potential.append(Member)
    if Potential != []:        
        Austerity_and_Potential.append(Potential)   


**Create compared List which entails names of economists and corresponding names of think tank boards**

In [30]:
Liste = pd.DataFrame(Austerity_and_Potential)
Liste = Liste.drop_duplicates()
Liste = Liste.drop(Liste.columns[[3,4,5]], axis=1)
Liste.columns = ['Oekonom', 'Emp name', 'JaroScore']
#Liste.to_csv('/Users/Waspish/Documents/WZB/ListComparison/Potential_MPS_Member.csv',encoding = "ISO-8859-1", sep=";")

Liste

,Oekonom,Emp name,JaroScore
0,Charles Blankart,Charles B.Blankart,1.000000
1,Lars P. Feld,Lars P.Feld,0.872960
2,Justus Haucap,JustusHaucap,0.992857
3,Kai Konrad,Kai A.Konrad,0.974359
4,Wolf Sch_fer,WolfSch_fer,0.948718
5,Friedrich Schneider,FriedrichSchneider,1.000000
6,Christian Seidl,ChristianSeidel,0.936111
7,Joachim Starbatty,JoachimStarbatty,0.962963
8,Thomas Straubhaar,ThomasStraubhaar,0.994444
9,Thusnelda Tivig,ThusneldaTivig,0.958333


**Check how many matches we get**

In [31]:
len(Liste.Oekonom)

13

**Rename the list**

In [32]:
Liste = Liste.merge(Austerity_Members, on='Emp name', how='left')
Liste.columns = ['Name', 'Oekonom','JaroScore','Thinktank']

**Merge both lists**

In [33]:
Liste = Liste.merge(dfname, on='Name', how='left')
Liste = Liste.drop(Liste.columns[[1,2,4,6]], axis=1)
Liste

,Name,Thinktank,Forschungseinrichtung,Title
0,Charles Blankart,Die Familienunternehmer - ASU,Humboldt-Universit_t Berlin,Prof.
1,Charles Blankart,Hayek Institute,Humboldt-Universit_t Berlin,Prof.
2,Lars P. Feld,Aktionsgemeinschaft Soziale Marktwirtschaft,Universit_t Marburg,Prof.
3,Justus Haucap,ECONWATCH,Universit_t Bochum,Prof.
4,Kai Konrad,ECONWATCH,Wissenschaftszentrum Berlin,Prof.
5,Wolf Sch_fer,Hamburg Institute for International Economics,Helmut-Schmidt-Universit_t Hamburg,Prof.
6,Friedrich Schneider,Agenda Austria,Universit_t Linz,Prof.
7,Christian Seidl,Hanns Seidel Stiftung,Universit_t Kiel,Prof.
8,Joachim Starbatty,Aktionsgemeinschaft Soziale Marktwirtschaft,Universit_t TÙbingen,Prof.
9,Thomas Straubhaar,Friedrich Naumann Foundation,Universit_t Hamburg und HWWI,Prof.


**Drop duplicates**

In [34]:
#Liste = Liste.Thinktank.append(Liste.Forschungseinrichtung)
Liste = Liste.drop_duplicates()
Liste

,Name,Thinktank,Forschungseinrichtung,Title
0,Charles Blankart,Die Familienunternehmer - ASU,Humboldt-Universit_t Berlin,Prof.
1,Charles Blankart,Hayek Institute,Humboldt-Universit_t Berlin,Prof.
2,Lars P. Feld,Aktionsgemeinschaft Soziale Marktwirtschaft,Universit_t Marburg,Prof.
3,Justus Haucap,ECONWATCH,Universit_t Bochum,Prof.
4,Kai Konrad,ECONWATCH,Wissenschaftszentrum Berlin,Prof.
5,Wolf Sch_fer,Hamburg Institute for International Economics,Helmut-Schmidt-Universit_t Hamburg,Prof.
6,Friedrich Schneider,Agenda Austria,Universit_t Linz,Prof.
7,Christian Seidl,Hanns Seidel Stiftung,Universit_t Kiel,Prof.
8,Joachim Starbatty,Aktionsgemeinschaft Soziale Marktwirtschaft,Universit_t TÙbingen,Prof.
9,Thomas Straubhaar,Friedrich Naumann Foundation,Universit_t Hamburg und HWWI,Prof.


# Comparison if Individual is in Mont Pelerin Society

Die Mont Pèlerin Society (MPS) ist ein 1947 von Friedrich von Hayek gegründeter Zusammenschluss von Akademikern, Geschäftsleuten und Journalisten, mit dem Ziel, zukünftige Generationen vom Wirtschaftsliberalismus zu überzeugen (Source: Wikipedia)

**MPS_Members is a list of all active and former Members of the Mont Pelerin Society**

In [40]:
MPS_Path = '~/Csv_Dateien/MPS_Members.csv'
MPS_Members = pd.read_csv(MPS_Path, encoding = "ISO-8859-1", sep=";")
MPS_Members.head()

,MPSMitglieder
0,Harry Burrows Acton
1,Wayne K. Adams
2,"William H. Adams, III"
3,Douglas K. Adie
4,Alfred (Fred) Aftalion


**check if indidivdual that signed Hamburger Appell holds membership in the mont pelerin society. 
Does not only account for direct matches but takes similar names into account (jaro_score).** 

In [41]:
Members_and_Potential = []

for name in Liste['Name']:
    Potential = []
    Member = []
    for mps in MPS_Members['MPSMitglieder']:
        distance = Levenshtein.distance(name,mps)
        jaro = Levenshtein.jaro_winkler(name,mps)
        if name == mps:
            Member.append(name)
            Member.append(mps)
            Member.append(jaro)
        
        elif distance <= 3 and distance != 0 and jaro>=0.80:
            Potential.append(name)
            Potential.append(mps)
            Potential.append(jaro)
    if Member != []:
        Members_and_Potential.append(Member)
    if Potential != []:        
        Members_and_Potential.append(Potential)   

Members_and_Potential = pd.DataFrame(Members_and_Potential) 

In [42]:
Members_and_Potential.columns = ['Name', 'Vergleich', 'JaroScore']
Members_and_Potential['Membership'] = 1


In [43]:
Members_and_Potential = Members_and_Potential.drop(Members_and_Potential.columns[[1,2]], axis=1)

**Merge dataframe of potential Members of the MPS dataframe of Professors that signed the Hamburger Appell which also hold a membership in a think tank**

In [44]:
Liste = Liste.merge(Members_and_Potential, on='Name', how='left')
Liste = Liste.fillna(0)
Liste['Appell'] = 'HamburgerAppell'
Liste

,Name,Thinktank,Forschungseinrichtung,Title,Membership_x,Appell,Membership_y
0,Charles Blankart,Die Familienunternehmer - ASU,Humboldt-Universit_t Berlin,Prof.,0.0,HamburgerAppell,0.0
1,Charles Blankart,Hayek Institute,Humboldt-Universit_t Berlin,Prof.,0.0,HamburgerAppell,0.0
2,Lars P. Feld,Aktionsgemeinschaft Soziale Marktwirtschaft,Universit_t Marburg,Prof.,1.0,HamburgerAppell,1.0
3,Justus Haucap,ECONWATCH,Universit_t Bochum,Prof.,0.0,HamburgerAppell,0.0
4,Kai Konrad,ECONWATCH,Wissenschaftszentrum Berlin,Prof.,0.0,HamburgerAppell,0.0
5,Wolf Sch_fer,Hamburg Institute for International Economics,Helmut-Schmidt-Universit_t Hamburg,Prof.,0.0,HamburgerAppell,0.0
6,Friedrich Schneider,Agenda Austria,Universit_t Linz,Prof.,0.0,HamburgerAppell,0.0
7,Christian Seidl,Hanns Seidel Stiftung,Universit_t Kiel,Prof.,0.0,HamburgerAppell,0.0
8,Joachim Starbatty,Aktionsgemeinschaft Soziale Marktwirtschaft,Universit_t TÙbingen,Prof.,1.0,HamburgerAppell,1.0
9,Thomas Straubhaar,Friedrich Naumann Foundation,Universit_t Hamburg und HWWI,Prof.,0.0,HamburgerAppell,0.0


**Save dataframe to csv file for further analysis**

In [45]:
Path = ''
Liste.to_csv(Path, encoding = "ISO-8859-1", sep=";", ignore_index=True)